## Capturing : Capture the geometry from the sampling and convert to file array ##
Maintain: Demo and test only. <br/>
For real calculation, a separate UNIX shell script that run the calculation and another separate Python script to capture the results (either directly from the .xyz file or summary from csv file) is required


In [1]:
test_name = "glycine_non_opt_350_20000s"

In [2]:
import os
import sys
import pandas as pd

sys.path.insert(0, "..")

In [3]:
from ase.calculators.nwchem import NWChem
os.environ['ASE_NWCHEM_COMMAND'] = 'mpirun -n 8 nwchem PREFIX.nw > PREFIX.out'
nwchem_calc = NWChem(label='calc/nwchem',
              maxiter=2000,
              xc='b3lyp',
              basis='6-31G**')



In [4]:
import numpy as np
from ase.units import Hartree, mol, kcal, Bohr
def nwchem_compute(md_samples_arr, nwchem_calc):
    """
    Take the array with the atoms, compute the DFT energy for all the given configurations.
    """
    
    md_dft_energy_arr = np.zeros(md_samples_arr.size + 1)
    md_dft_force_arr = pd.Series([]) # Use pd.Series since molecule size may vary
    for index, atoms in md_samples_arr.items():
        atoms.set_calculator(nwchem_calc)
        potent_e = atoms.get_potential_energy()
        force_arr = atoms.get_forces()
        print(index)
        print(potent_e)
        md_dft_energy_arr[index] = potent_e
        md_dft_force_arr[index] = force_arr
    
    return md_dft_energy_arr, md_dft_force_arr

def get_energy_force_mol(atoms, calc):
    """Get the energy for each molecule for a particular molecule (atoms)
    
            Args:
                atoms: an ase.Atoms object
                calc: an ase.calculator
                
            Outputs:
                poten_e*: potential energy for the molecule
                force_arr*: force for the molecule
                
                * Both wrapped in a Pandas.Series() object
    
    Comments:
    1. For FileIO Calculator, May improve the performance by 
    getting all the character at once.
    
    """
    atoms.set_calculator(calc)
    poten_e = atoms.get_potential_energy()
    force_arr = atoms.get_forces()
    
    return pd.Series([poten_e, force_arr])
    
# def dft_compute(md_samples_arr, dft_calc):
#     """Compute the DFT Energy for the given array using the given calculator
    
#             Args:
#                 md_samplesArr: a pd.Series() of all the configurations 
#                 dft_calc:  a ase.calculator that will be assigned to the atoms
                
#             Outputs:
#                 md_dft_energy_arr: an numpy array of all the dft energy
    
#     Comments: 
#     1. Attempt to generalize in the naming.
#     2. Attempt to use element-wise operations in pandas.Series(), 
#     which may not work for smaller sets.
#     3. Untested. 
#     """
    
#     md_dft_energy_arr = np.zeros(md_samples_arr.size)
#     md_samples_arr.apply(

In [5]:
## Global Var:
relative_energy_ref_dft = 0
relative_energy_ref_dftb = 0

def get_relative_energy(energy_arr):
    """
    Get the relative energy with reference to the lowest point in all the energy. 
    
        Args:
            energy_arr: an np.array of energy (either dftb or dft) of shape (No. samples, )
    
    """
    lowest_energy = np.amin(energy_arr)
    relative_energy_arr = energy_arr - lowest_energy
    
    return relative_energy_arr, lowest_energy


In [6]:
from dftb_traj_io import read_dftb_atom_from_traj
from dftb_traj_io import read_dftb_md_energy

## Get the Hybrid Rotating Sample Training Set 
Combine the MD trajectory with different initial geometries rotated for 60 degree. 

In [7]:
##########
# WARNING: TEMPORARY!
# Use it only because the original file has been lost. 
##########
sample_file_list = [ 'glycine_190HOCO_400K_10000s', 'glycine_360HOCO_400K_10000s']
md_samplesArr = pd.Series([])
for sample_file in sample_file_list:
    sample_file_name = '../md_sampling/samples/' + sample_file  + '/geom.out.xyz'
    md_samplesArr_temp, md_Mulliken = read_dftb_atom_from_traj(sample_file_name, writeTrajectory=True)
    md_samplesArr = md_samplesArr.append(md_samplesArr_temp, ignore_index=True)
    print(md_Mulliken.shape)
print(md_samplesArr.shape)

(10001, 10)
(10001, 10)
(20002,)


In [8]:
import os 
os.getcwd()


'/home/junmian/Applications/YeltsinNet/md_capture'

In [9]:
md_samplesArr

0        (Atom('C', [0.622, -0.1192, 0.0383], index=0),...
1        (Atom('C', [0.62255955, -0.1199231, 0.03914185...
2        (Atom('C', [0.62307313, -0.12065897, 0.0399874...
3        (Atom('C', [0.62354007, -0.12140603, 0.0408360...
4        (Atom('C', [0.6239599, -0.12216273, 0.0416871]...
5        (Atom('C', [0.62433229, -0.12292757, 0.0425398...
6        (Atom('C', [0.62465714, -0.12369907, 0.0433934...
7        (Atom('C', [0.62493447, -0.12447582, 0.0442472...
8        (Atom('C', [0.62516454, -0.12525644, 0.0451004...
9        (Atom('C', [0.62534773, -0.12603961, 0.0459522...
10       (Atom('C', [0.62548464, -0.12682406, 0.0468016...
11       (Atom('C', [0.625576, -0.12760858, 0.04764779]...
12       (Atom('C', [0.62562273, -0.12839202, 0.0484898...
13       (Atom('C', [0.62562591, -0.12917327, 0.0493268...
14       (Atom('C', [0.62558676, -0.12995128, 0.0501577...
15       (Atom('C', [0.62550666, -0.13072507, 0.0509816...
16       (Atom('C', [0.62538712, -0.1314937, 0.05179754.

In [10]:
sys.path.insert(0, "../Calculator")

## Training Notice:
Select 3000 configurations from the sampling and run the calculation. 

In [11]:
n_samples = len(md_samplesArr) - 1

n_train = 1000 

train_idx = np.linspace(0, n_samples, n_train, dtype=np.int32)
print(train_idx.shape)

(1000,)


In [12]:
print(train_idx)

[    0    20    40    60    80   100   120   140   160   180   200   220
   240   260   280   300   320   340   360   380   400   420   440   460
   480   500   520   540   560   580   600   620   640   660   680   700
   720   740   760   780   800   820   840   860   880   900   920   940
   961   981  1001  1021  1041  1061  1081  1101  1121  1141  1161  1181
  1201  1221  1241  1261  1281  1301  1321  1341  1361  1381  1401  1421
  1441  1461  1481  1501  1521  1541  1561  1581  1601  1621  1641  1661
  1681  1701  1721  1741  1761  1781  1801  1821  1841  1861  1881  1901
  1922  1942  1962  1982  2002  2022  2042  2062  2082  2102  2122  2142
  2162  2182  2202  2222  2242  2262  2282  2302  2322  2342  2362  2382
  2402  2422  2442  2462  2482  2502  2522  2542  2562  2582  2602  2622
  2642  2662  2682  2702  2722  2742  2762  2782  2802  2822  2842  2863
  2883  2903  2923  2943  2963  2983  3003  3023  3043  3063  3083  3103
  3123  3143  3163  3183  3203  3223  3243  3263  3

In [13]:
md_train_arr = md_samplesArr[train_idx]
print(md_train_arr.shape)
print(type(md_train_arr))

(1000,)
<class 'pandas.core.series.Series'>


## Calculate Energy 
Calculate the Energy using DFTB and Higher-Level Quantum Chemistry Potential. 

In [14]:
# def calc_compute(md_samples_arr, nwchem_calc, train_idx):
#     """
#     Take the array with the atoms, compute the DFT energy for all the given configurations.
    
#     (Seems pandas series has some problem with indexing )
#     """
    
#     md_dft_energy_arr = np.zeros(md_samples_arr.size + 1)
#     md_dft_force_arr = pd.Series([]) # Use pd.Series since molecule size may vary
#     for index, atoms in m:
#         atoms.set_calculator(nwchem_calc)
#         potent_e = atoms.get_potential_energy()
#         force_arr = atoms.get_forces()
#         print(index)
#         print(potent_e)
#         md_dft_energy_arr[index] = potent_e
#         md_dft_force_arr[index] = force_arr
    
#     return md_dft_energy_arr, md_dft_force_arr

def nwchem_compute(md_samples_arr, nwchem_calc):
    """
    Take the array with the atoms, compute the DFT energy for all the given configurations.
    """
    
    md_dft_energy_arr = np.zeros(md_samples_arr.size)
    md_dft_force_arr = pd.Series([]) # Use pd.Series since molecule size may vary
    count = 0
    for index, atoms in md_samples_arr.items():
        atoms.set_calculator(nwchem_calc)
        potent_e = atoms.get_potential_energy()
        force_arr = atoms.get_forces()
        print('Index:', index, ' Count:',  count, " Potential Energy: ", potent_e)
        md_dft_energy_arr[count] = potent_e
        md_dft_force_arr[count] = force_arr
        count += 1
    
    return md_dft_energy_arr, md_dft_force_arr


In [15]:
from ASEDFTBMod import DftbMod
os.environ['DFTB_PREFIX'] = '/home/junmian/Applications/dftbplus/mio-1-1/'
os.environ['DFTB_COMMAND'] = '/home/junmian/Applications/dftbplus/_install/bin/dftb+'


# Set path to the model


dftb_calc = DftbMod(
                        Hamiltonian_SCC ='YES',
                        Hamiltonian_SCCTolerance = 1.0E-008,
                        Hamiltonian_MaxAngularMomentum_='',
                        Hamiltonian_MaxAngularMomentum_O='"p"',
                        Hamiltonian_MaxAngularMomentum_H='"s"',
                        Hamiltonian_MaxAngularMomentum_C='"p"',
                        Hamiltonian_MaxAngularMomentum_N='"p"',
                         )


In [16]:
#md_energy_arr, md_kinetic_energy_arr = read_dftb_md_energy('../md_sampling/md.out')
md_energy_arr, dft_force_arr = nwchem_compute(md_train_arr, dftb_calc)
md_energy_arr.shape
print(md_energy_arr)

Index: 0  Count: 0  Potential Energy:  -390.06926171383833
Index: 20  Count: 1  Potential Energy:  -389.96635259219187
Index: 40  Count: 2  Potential Energy:  -389.9381830110273
Index: 60  Count: 3  Potential Energy:  -389.96710863732494
Index: 80  Count: 4  Potential Energy:  -389.907714982098
Index: 100  Count: 5  Potential Energy:  -389.92383917797684
Index: 120  Count: 6  Potential Energy:  -390.0092806328782
Index: 140  Count: 7  Potential Energy:  -389.9587645570957
Index: 160  Count: 8  Potential Energy:  -389.9345118629921
Index: 180  Count: 9  Potential Energy:  -389.9362126856166
Index: 200  Count: 10  Potential Energy:  -389.9755518397462
Index: 220  Count: 11  Potential Energy:  -389.98230716842113
Index: 240  Count: 12  Potential Energy:  -389.90863035848685
Index: 260  Count: 13  Potential Energy:  -389.942467157191
Index: 280  Count: 14  Potential Energy:  -389.9707100825268
Index: 300  Count: 15  Potential Energy:  -389.91662516091895
Index: 320  Count: 16  Potential En

Index: 2702  Count: 135  Potential Energy:  -389.90960508948564
Index: 2722  Count: 136  Potential Energy:  -389.95090699741974
Index: 2742  Count: 137  Potential Energy:  -389.97320682198267
Index: 2762  Count: 138  Potential Energy:  -389.9594151144227
Index: 2782  Count: 139  Potential Energy:  -389.9742022196264
Index: 2802  Count: 140  Potential Energy:  -389.92046282474786
Index: 2822  Count: 141  Potential Energy:  -389.93667439361707
Index: 2842  Count: 142  Potential Energy:  -389.99547885066175
Index: 2863  Count: 143  Potential Energy:  -389.9383261174915
Index: 2883  Count: 144  Potential Energy:  -389.8994795432785
Index: 2903  Count: 145  Potential Energy:  -389.9236291465618
Index: 2923  Count: 146  Potential Energy:  -389.89047957182527
Index: 2943  Count: 147  Potential Energy:  -389.91635334429685
Index: 2963  Count: 148  Potential Energy:  -389.9851330663617
Index: 2983  Count: 149  Potential Energy:  -389.899784788438
Index: 3003  Count: 150  Potential Energy:  -389

Index: 5365  Count: 268  Potential Energy:  -389.9486656337331
Index: 5385  Count: 269  Potential Energy:  -389.8477670227417
Index: 5405  Count: 270  Potential Energy:  -389.91602498437106
Index: 5425  Count: 271  Potential Energy:  -389.92079081313483
Index: 5445  Count: 272  Potential Energy:  -389.89859447418036
Index: 5465  Count: 273  Potential Energy:  -389.9198818406708
Index: 5485  Count: 274  Potential Energy:  -389.8321572156469
Index: 5505  Count: 275  Potential Energy:  -389.88766846507167
Index: 5525  Count: 276  Potential Energy:  -389.90522743130674
Index: 5545  Count: 277  Potential Energy:  -389.8891927430264
Index: 5565  Count: 278  Potential Energy:  -389.9056937066976
Index: 5585  Count: 279  Potential Energy:  -389.91460461157544
Index: 5605  Count: 280  Potential Energy:  -389.92545718377653
Index: 5625  Count: 281  Potential Energy:  -389.86607971377316
Index: 5645  Count: 282  Potential Energy:  -389.93289680913523
Index: 5665  Count: 283  Potential Energy:  -3

Index: 7988  Count: 399  Potential Energy:  -389.9201934852517
Index: 8008  Count: 400  Potential Energy:  -389.96485516577434
Index: 8028  Count: 401  Potential Energy:  -389.96519698596734
Index: 8048  Count: 402  Potential Energy:  -389.9051883716397
Index: 8068  Count: 403  Potential Energy:  -390.00678735291194
Index: 8088  Count: 404  Potential Energy:  -390.000382405464
Index: 8108  Count: 405  Potential Energy:  -389.89225957857354
Index: 8128  Count: 406  Potential Energy:  -389.9634034694407
Index: 8148  Count: 407  Potential Energy:  -389.92978331291835
Index: 8168  Count: 408  Potential Energy:  -389.9147489680654
Index: 8188  Count: 409  Potential Energy:  -389.96159451884995
Index: 8208  Count: 410  Potential Energy:  -389.9306675704342
Index: 8228  Count: 411  Potential Energy:  -389.99594705633586
Index: 8248  Count: 412  Potential Energy:  -389.9532382574846
Index: 8268  Count: 413  Potential Energy:  -389.91689970850297
Index: 8288  Count: 414  Potential Energy:  -389

Index: 10591  Count: 529  Potential Energy:  -389.84165426160297
Index: 10611  Count: 530  Potential Energy:  -389.79858470948324
Index: 10631  Count: 531  Potential Energy:  -389.8311898154853
Index: 10651  Count: 532  Potential Energy:  -389.7663821752622
Index: 10671  Count: 533  Potential Energy:  -389.86510800105594
Index: 10691  Count: 534  Potential Energy:  -389.9679590081915
Index: 10711  Count: 535  Potential Energy:  -389.8233583244559
Index: 10731  Count: 536  Potential Energy:  -389.7567882078349
Index: 10751  Count: 537  Potential Energy:  -389.80469392067084
Index: 10771  Count: 538  Potential Energy:  -389.8341715173677
Index: 10791  Count: 539  Potential Energy:  -389.86818935567345
Index: 10811  Count: 540  Potential Energy:  -389.82951326420897
Index: 10831  Count: 541  Potential Energy:  -389.79479202744415
Index: 10851  Count: 542  Potential Energy:  -389.7930351985109
Index: 10871  Count: 543  Potential Energy:  -389.86250194121726
Index: 10891  Count: 544  Potent

Index: 13173  Count: 658  Potential Energy:  -389.79098226936776
Index: 13193  Count: 659  Potential Energy:  -389.9465192171712
Index: 13213  Count: 660  Potential Energy:  -389.81395541640677
Index: 13233  Count: 661  Potential Energy:  -389.8074176562245
Index: 13253  Count: 662  Potential Energy:  -389.90021466464896
Index: 13273  Count: 663  Potential Energy:  -389.7765322840582
Index: 13293  Count: 664  Potential Energy:  -389.81169598283606
Index: 13313  Count: 665  Potential Energy:  -389.84645625569743
Index: 13334  Count: 666  Potential Energy:  -389.84144169702836
Index: 13354  Count: 667  Potential Energy:  -389.8450770997651
Index: 13374  Count: 668  Potential Energy:  -389.87237427117236
Index: 13394  Count: 669  Potential Energy:  -389.83067352872075
Index: 13414  Count: 670  Potential Energy:  -389.7923265765107
Index: 13434  Count: 671  Potential Energy:  -389.78588620133206
Index: 13454  Count: 672  Potential Energy:  -389.8302654369196
Index: 13474  Count: 673  Poten

Index: 15816  Count: 790  Potential Energy:  -389.8126844149173
Index: 15836  Count: 791  Potential Energy:  -389.7012230359221
Index: 15856  Count: 792  Potential Energy:  -389.8605630062138
Index: 15876  Count: 793  Potential Energy:  -389.93905877675394
Index: 15896  Count: 794  Potential Energy:  -389.81733610009377
Index: 15916  Count: 795  Potential Energy:  -389.85388350938314
Index: 15936  Count: 796  Potential Energy:  -389.78938859705886
Index: 15956  Count: 797  Potential Energy:  -389.7901212658181
Index: 15976  Count: 798  Potential Energy:  -389.81987289756705
Index: 15996  Count: 799  Potential Energy:  -389.84300761412896
Index: 16016  Count: 800  Potential Energy:  -389.86094466386777
Index: 16036  Count: 801  Potential Energy:  -389.76621307530655
Index: 16056  Count: 802  Potential Energy:  -389.83260778062834
Index: 16076  Count: 803  Potential Energy:  -389.834989091246
Index: 16096  Count: 804  Potential Energy:  -389.83528343004116
Index: 16116  Count: 805  Poten

Index: 18379  Count: 918  Potential Energy:  -389.7901834206946
Index: 18399  Count: 919  Potential Energy:  -389.91993077599477
Index: 18419  Count: 920  Potential Energy:  -389.9017415144334
Index: 18439  Count: 921  Potential Energy:  -389.82628947754785
Index: 18459  Count: 922  Potential Energy:  -389.8867345317339
Index: 18479  Count: 923  Potential Energy:  -389.80147444837496
Index: 18499  Count: 924  Potential Energy:  -389.7789029494755
Index: 18519  Count: 925  Potential Energy:  -389.832766159638
Index: 18539  Count: 926  Potential Energy:  -389.84770641336854
Index: 18559  Count: 927  Potential Energy:  -389.77023171653457
Index: 18579  Count: 928  Potential Energy:  -389.7931232371218
Index: 18599  Count: 929  Potential Energy:  -389.85305095801783
Index: 18619  Count: 930  Potential Energy:  -389.80700484951194
Index: 18639  Count: 931  Potential Energy:  -389.8918304599929
Index: 18659  Count: 932  Potential Energy:  -389.8979352439751
Index: 18679  Count: 933  Potentia

In [17]:
md_dft_energy_arr, md_dft_force_arr = nwchem_compute(md_train_arr, nwchem_calc)

Index: 0  Count: 0  Potential Energy:  -7739.7898574371375
Index: 20  Count: 1  Potential Energy:  -7739.523221207866
Index: 40  Count: 2  Potential Energy:  -7739.450937374883
Index: 60  Count: 3  Potential Energy:  -7739.415490179201
Index: 80  Count: 4  Potential Energy:  -7739.285012088992
Index: 100  Count: 5  Potential Energy:  -7739.472173067366
Index: 120  Count: 6  Potential Energy:  -7739.553548659964
Index: 140  Count: 7  Potential Energy:  -7739.522492273774
Index: 160  Count: 8  Potential Energy:  -7739.549152597504
Index: 180  Count: 9  Potential Energy:  -7739.551879511803
Index: 200  Count: 10  Potential Energy:  -7739.559513109626
Index: 220  Count: 11  Potential Energy:  -7739.509262897442
Index: 240  Count: 12  Potential Energy:  -7739.428587252958
Index: 260  Count: 13  Potential Energy:  -7739.411713298767
Index: 280  Count: 14  Potential Energy:  -7739.495950657322
Index: 300  Count: 15  Potential Energy:  -7739.446511226236
Index: 320  Count: 16  Potential Energy

Index: 2662  Count: 133  Potential Energy:  -7739.211510904287
Index: 2682  Count: 134  Potential Energy:  -7739.33281702698
Index: 2702  Count: 135  Potential Energy:  -7739.288001841661
Index: 2722  Count: 136  Potential Energy:  -7739.365382513672
Index: 2742  Count: 137  Potential Energy:  -7739.525513170148
Index: 2762  Count: 138  Potential Energy:  -7739.468086921203
Index: 2782  Count: 139  Potential Energy:  -7739.504239386297
Index: 2802  Count: 140  Potential Energy:  -7739.433194689955
Index: 2822  Count: 141  Potential Energy:  -7739.470172547164
Index: 2842  Count: 142  Potential Energy:  -7739.502370267621
Index: 2863  Count: 143  Potential Energy:  -7739.4667258368445
Index: 2883  Count: 144  Potential Energy:  -7739.373813301836
Index: 2903  Count: 145  Potential Energy:  -7739.307662621434
Index: 2923  Count: 146  Potential Energy:  -7739.38148926599
Index: 2943  Count: 147  Potential Energy:  -7739.355118180926
Index: 2963  Count: 148  Potential Energy:  -7739.473397

Index: 5265  Count: 263  Potential Energy:  -7739.402449909184
Index: 5285  Count: 264  Potential Energy:  -7739.537048811365
Index: 5305  Count: 265  Potential Energy:  -7739.601290499374
Index: 5325  Count: 266  Potential Energy:  -7739.306101887763
Index: 5345  Count: 267  Potential Energy:  -7739.272059805795
Index: 5365  Count: 268  Potential Energy:  -7739.423568929855
Index: 5385  Count: 269  Potential Energy:  -7739.31561056595
Index: 5405  Count: 270  Potential Energy:  -7739.47410545012
Index: 5425  Count: 271  Potential Energy:  -7739.44813750498
Index: 5445  Count: 272  Potential Energy:  -7739.292124054692
Index: 5465  Count: 273  Potential Energy:  -7739.408217565367
Index: 5485  Count: 274  Potential Energy:  -7739.3286268740285
Index: 5505  Count: 275  Potential Energy:  -7739.400117759466
Index: 5525  Count: 276  Potential Energy:  -7739.456518677237
Index: 5545  Count: 277  Potential Energy:  -7739.395542529131
Index: 5565  Count: 278  Potential Energy:  -7739.3571158

Index: 7888  Count: 394  Potential Energy:  -7739.506832242127
Index: 7908  Count: 395  Potential Energy:  -7739.38479870333
Index: 7928  Count: 396  Potential Energy:  -7739.3840892312555
Index: 7948  Count: 397  Potential Energy:  -7739.551814276615
Index: 7968  Count: 398  Potential Energy:  -7739.426272073378
Index: 7988  Count: 399  Potential Energy:  -7739.464657685818
Index: 8008  Count: 400  Potential Energy:  -7739.554552153784
Index: 8028  Count: 401  Potential Energy:  -7739.481993098912
Index: 8048  Count: 402  Potential Energy:  -7739.493888080548
Index: 8068  Count: 403  Potential Energy:  -7739.505968698832
Index: 8088  Count: 404  Potential Energy:  -7739.405688455312
Index: 8108  Count: 405  Potential Energy:  -7739.316710088289
Index: 8128  Count: 406  Potential Energy:  -7739.472871499697
Index: 8148  Count: 407  Potential Energy:  -7739.453711231955
Index: 8168  Count: 408  Potential Energy:  -7739.392585590562
Index: 8188  Count: 409  Potential Energy:  -7739.46221

Index: 10491  Count: 524  Potential Energy:  -7739.415752156275
Index: 10511  Count: 525  Potential Energy:  -7739.279483603503
Index: 10531  Count: 526  Potential Energy:  -7739.422180497428
Index: 10551  Count: 527  Potential Energy:  -7739.423858758354
Index: 10571  Count: 528  Potential Energy:  -7739.531910261147
Index: 10591  Count: 529  Potential Energy:  -7739.591316554188
Index: 10611  Count: 530  Potential Energy:  -7739.46853955352
Index: 10631  Count: 531  Potential Energy:  -7739.554117390776
Index: 10651  Count: 532  Potential Energy:  -7739.407801235678
Index: 10671  Count: 533  Potential Energy:  -7739.579232968991
Index: 10691  Count: 534  Potential Energy:  -7739.655011709862
Index: 10711  Count: 535  Potential Energy:  -7739.521584121848
Index: 10731  Count: 536  Potential Energy:  -7739.571607181818
Index: 10751  Count: 537  Potential Energy:  -7739.432936958129
Index: 10771  Count: 538  Potential Energy:  -7739.5313323711725
Index: 10791  Count: 539  Potential Ener

Index: 13053  Count: 652  Potential Energy:  -7739.438243301971
Index: 13073  Count: 653  Potential Energy:  -7739.6745168318275
Index: 13093  Count: 654  Potential Energy:  -7739.584210694367
Index: 13113  Count: 655  Potential Energy:  -7739.5290689952135
Index: 13133  Count: 656  Potential Energy:  -7739.650115436275
Index: 13153  Count: 657  Potential Energy:  -7739.438926552418
Index: 13173  Count: 658  Potential Energy:  -7739.54295900008
Index: 13193  Count: 659  Potential Energy:  -7739.617352195108
Index: 13213  Count: 660  Potential Energy:  -7739.444224296917
Index: 13233  Count: 661  Potential Energy:  -7739.482151632501
Index: 13253  Count: 662  Potential Energy:  -7739.529440847484
Index: 13273  Count: 663  Potential Energy:  -7739.564526323139
Index: 13293  Count: 664  Potential Energy:  -7739.5238232205975
Index: 13313  Count: 665  Potential Energy:  -7739.622997921423
Index: 13334  Count: 666  Potential Energy:  -7739.57504443482
Index: 13354  Count: 667  Potential Ene

Index: 15636  Count: 781  Potential Energy:  -7739.506597447919
Index: 15656  Count: 782  Potential Energy:  -7739.5087729681645
Index: 15676  Count: 783  Potential Energy:  -7739.765692530434
Index: 15696  Count: 784  Potential Energy:  -7739.553879727837
Index: 15716  Count: 785  Potential Energy:  -7739.532926998122
Index: 15736  Count: 786  Potential Energy:  -7739.486401767264
Index: 15756  Count: 787  Potential Energy:  -7739.382694463719
Index: 15776  Count: 788  Potential Energy:  -7739.55506912952
Index: 15796  Count: 789  Potential Energy:  -7739.626312673391
Index: 15816  Count: 790  Potential Energy:  -7739.58912109597
Index: 15836  Count: 791  Potential Energy:  -7739.419610980749
Index: 15856  Count: 792  Potential Energy:  -7739.645265783138
Index: 15876  Count: 793  Potential Energy:  -7739.610476132557
Index: 15896  Count: 794  Potential Energy:  -7739.462112062125
Index: 15916  Count: 795  Potential Energy:  -7739.619358216009
Index: 15936  Count: 796  Potential Energ

Index: 18199  Count: 909  Potential Energy:  -7739.612965832579
Index: 18219  Count: 910  Potential Energy:  -7739.5113851977385
Index: 18239  Count: 911  Potential Energy:  -7739.292389057402
Index: 18259  Count: 912  Potential Energy:  -7739.452760045983
Index: 18279  Count: 913  Potential Energy:  -7739.4165532468805
Index: 18299  Count: 914  Potential Energy:  -7739.527914563645
Index: 18319  Count: 915  Potential Energy:  -7739.569065466822
Index: 18339  Count: 916  Potential Energy:  -7739.359818253754
Index: 18359  Count: 917  Potential Energy:  -7739.5021703162065
Index: 18379  Count: 918  Potential Energy:  -7739.424340915869
Index: 18399  Count: 919  Potential Energy:  -7739.547337749569
Index: 18419  Count: 920  Potential Energy:  -7739.58482565961
Index: 18439  Count: 921  Potential Energy:  -7739.566217719056
Index: 18459  Count: 922  Potential Energy:  -7739.637897496875
Index: 18479  Count: 923  Potential Energy:  -7739.4632647576345
Index: 18499  Count: 924  Potential E

In [18]:
md_dft_rel_e_arr, dft_ref = get_relative_energy(md_dft_energy_arr)
print(dft_ref)
print(md_dft_rel_e_arr)

-7739.814734543828
[0.02487711 0.29151334 0.36379717 0.39924436 0.52972245 0.34256148
 0.26118588 0.29224227 0.26558195 0.26285503 0.25522143 0.30547165
 0.38614729 0.40302125 0.31878389 0.36822332 0.33675803 0.25357734
 0.4088559  0.4181073  0.38084458 0.5078916  0.46660074 0.42281566
 0.43528987 0.45609609 0.46372525 0.38531281 0.34342659 0.3130889
 0.27851025 0.29916765 0.4618911  0.41516013 0.35850528 0.55009071
 0.34110637 0.25932809 0.37518284 0.2425043  0.28177605 0.43410472
 0.41363976 0.3916573  0.4971358  0.41548364 0.36158679 0.46572975
 0.33848347 0.42228751 0.36679259 0.30531854 0.49302417 0.55245167
 0.57340984 0.55088604 0.38641004 0.21079432 0.32752846 0.42379663
 0.33048741 0.41639053 0.41629023 0.3811364  0.46755888 0.51413013
 0.47172701 0.31091417 0.32620759 0.319208   0.18915241 0.31746887
 0.44985289 0.43510875 0.37417439 0.39052532 0.25210025 0.22619416
 0.47320355 0.34265306 0.29707456 0.34895224 0.33477734 0.48988932
 0.42612596 0.35080698 0.32702879 0.33230838

In [19]:
#md_dft_rel_energy_arr, relative_energy_ref_dft= get_relative_energy(md_dft_energy_arr)
#md_dftb_rel_e_arr, relative_energy_ref_dftb = get_relative_energy(md_energy_arr)

dft_min_idx    = np.argmin(md_dft_energy_arr)

relative_energy_ref_dftb  = md_energy_arr[dft_min_idx]
md_dftb_rel_e_arr = md_energy_arr - relative_energy_ref_dftb



#print(relative_energy_ref_dft)
print(relative_energy_ref_dftb)
print(md_dftb_rel_e_arr)
#print(md_dft_rel_energy_arr)

-389.8730975617669
[-0.19616415 -0.09325503 -0.06508545 -0.09401108 -0.03461742 -0.05074162
 -0.13618307 -0.085667   -0.0614143  -0.06311512 -0.10245428 -0.10920961
 -0.0355328  -0.0693696  -0.09761252 -0.0435276  -0.09694601 -0.14725219
 -0.03088346  0.00852003 -0.1327087  -0.04837875 -0.023412   -0.06953277
  0.00901826  0.00318072 -0.07092132 -0.07478662 -0.03959499 -0.04336587
 -0.10188927 -0.09634143 -0.03482837 -0.05178193 -0.0593108   0.0260211
 -0.04733974 -0.11797699 -0.00653282 -0.04914324 -0.10065819  0.00583292
 -0.00320964 -0.03405222 -0.02056595 -0.0558511  -0.11175079 -0.05766347
 -0.00161025  0.00733806 -0.04199365 -0.10379455 -0.04562705 -0.02309961
 -0.03565693  0.01711847 -0.06205097 -0.09601043  0.00076961 -0.00255759
 -0.1174164  -0.09139374 -0.03350987 -0.0776955  -0.03063322  0.01385228
 -0.05931279 -0.09794958 -0.0614785  -0.03726416 -0.09593614 -0.06809733
  0.00656059 -0.02079059 -0.09443968 -0.06158292 -0.10052248 -0.15105263
  0.00666632  0.00690057 -0.12796

## Save the Bias ##

In [20]:
import pandas as pd
relative_energy = pd.Series([])
relative_energy['dft'] = dft_ref #relative_energy_ref_dft
relative_energy['calc'] = relative_energy_ref_dftb
#relative_energy.to_pickle('../relative_energy.pkl')

In [21]:
import WriteTrain
#test_name = 'rotating_120_180_append'

In [22]:
WriteTrain.write_train_samples(test_name, md_samplesArr, md_train_arr,
                               md_dftb_rel_e_arr, md_dft_rel_e_arr, 
                  relative_energy, 
                  md_energy_arr, md_dft_energy_arr, 
                  np.zeros( (1,1) ,dtype=np.float32), 
                    dft_force_arr, md_dft_force_arr)


md_samples_arr saved to  /home/junmian/Applications/YeltsinNet/md_capture/samples/glycine_non_opt_350_20000s/md_samples_arr.pkl
md_low_rel_energy_arr saved to  /home/junmian/Applications/YeltsinNet/md_capture/samples/glycine_non_opt_350_20000s/md_low_rel_e_arr_SCC.txt
rel_e_ref saved to /home/junmian/Applications/YeltsinNet/md_capture/samples/glycine_non_opt_350_20000s/rel_e_ref.pkl
md_high_rel_energy_arr saved to  /home/junmian/Applications/YeltsinNet/md_capture/samples/glycine_non_opt_350_20000s/md_high_rel_e_arr.txt


In [23]:
WriteTrain.write_train_samples

<function WriteTrain.write_train_samples(train_name, md_samples_arr, md_train_arr, md_low_rel_e_arr, md_high_rel_e_arr, rel_e_ref, md_low_e_arr, md_high_e_arr, md_Mulliken, md_low_force_arr, md_high_force_arr)>

In [24]:
#md_train_arr.to_pickle("/home/junmian/Applications/YeltsinNet/md_capture/samples/rotating_120_180_append/md_train_arr.pkl")

In [25]:
md_train_arr.to_pickle("/home/junmian/Applications/YeltsinNet/md_capture/samples/" + test_name + "/md_train_arr.pkl")